# Imports and settings

In [1]:
from openmm.app import *
from openmm import *                    
from openmm.unit import *

# Set up simulation + state

In [2]:
sim_temp = 300.0 * kelvin
H_mass = 4.0 * amu #Might need to be tuned to 3.5 amu 
time_step = 0.002 * picosecond  
nb_cutoff = 10.0 * angstrom                                                                                                          
box_padding = 12.0 * angstrom
salt_conc = 0.15 * molar
receptor_path="../villin.pdb"
current_file="villin-solvated"
# Misc parameters                                                 
restraint_distance = 0.0 * angstroms                              
restart_freq = 10
log_freq = 1                                                                                                                         
prd_steps = 100

In [3]:
# Load an already solvated PDB file and set up the system + state
pdb = PDBFile("../villin.pdb")
omm_forcefield = ForceField("amber/ff14SB.xml", "amber14/tip3p.xml")
system = omm_forcefield.createSystem(pdb.topology,
                                         nonbondedMethod=PME,
                                         nonbondedCutoff=10.0 * angstrom,
                                         constraints=HBonds,
                                         rigidWater=True,
                                         hydrogenMass=4.0 * amu)

# Setup the collective variables for the metadynamics simulation

For example's sake we will do the metadynamics simulation using 3 CVs. Two distances and\
and a dihedral angle. This atom indices are defined below for this case:  

In [4]:
# Define three atom indices - these will be used to measure useful distances 
d1_atom1_ind = 83
d1_atom2_ind = 151
d2_atom2_ind = 254

And now we define the dihedral angle atoms for the CV:

In [5]:
# Let's define a torsion we will use later
phe_chi = [72, 74, 76, 79]

Normally, to decide on the magnitude of the boosts, the "best practice" is to load in the\
CVs from a previous simulation and plot a histogram of the CVs. The height of the histogram\
is the height of the boost gaussian (used below as `height`).\
The width at half-max of the boost is the width of the gaussian (defined in the `BiasVariable`).

For simplicity, we've analyzed and defined the boost parameters already.\
We can define them explicitly when we define our CVs.\
Ultimately each CV is defined as a `BiasVariable` object.

In [6]:
# Define BiasVariables - Note that I've set the distances to be "Periodic=True"
# This allows us to use Mixed periodic and Non-periodic variables
from numpy import pi
chi = CustomTorsionForce('theta')
chi.addTorsion(phe_chi[0], phe_chi[1], phe_chi[2], phe_chi[3])
chi_bias = BiasVariable(chi, -pi, pi, 0.35, True, 181)

# Distance variables
d1 = CustomBondForce('r')
d1.addBond(d1_atom1_ind, d1_atom2_ind)
d1_bias = BiasVariable(d1, 0.3, 5.0, 0.05, True)

d2=CustomBondForce('r')
d2.addBond(d1_atom1_ind, d2_atom2_ind)
d2_bias = BiasVariable(d2, 0.3, 5.0, 0.05, True)


# Setup a fresh MetaD simulation

We define the Metadynamics simulation object below. Note that our step frequency 
here is *incredibly small* and should be set at *much higher* values.\
You should set frequency high enough that it will have time to diffuse away from the last Gaussian before adding another one.


In [7]:
# the meta object
meta = Metadynamics(system, [d1_bias, d2_bias, chi_bias], 300, 
                    biasFactor=5, height=1, frequency=10, 
                    saveFrequency=10,
                    biasDir="./")

In [8]:
#Define an integrator
integrator = LangevinMiddleIntegrator(300*kelvin, 1.0/picosecond, 0.002*picosecond)

In [9]:
simulation = Simulation(pdb.topology, system, integrator)

When running the situation, aside from using a `reporter` to save the trajectory, we can\
also use the `meta` object to extract the CV values directly. We can then print those out\
and plot them to see how the simulation is progressing.

Separately, the `saveFrequency` defined in the `meta` object defines how often the\
biases are saved to disk. This is useful if you want to stop the simulation. You can\
then load the biases back in and continue the simulation.

You can also use `meta.getFreeEnergy()` to generate a matrix of the FES as metadynamics\
is running.

In [10]:
simulation.context.setPositions(pdb.positions)
for i in range(5):
    meta.step(simulation, steps=10)
    n_steps = str(simulation.context.getStepCount())
    d1,d2,chi1 =meta._force.getCollectiveVariableValues(simulation.context)
    print("%s %s %s %s \n" % (n_steps, str(d1), str(d2), str(chi1)))

10 0.45591458678245544 0.42221570014953613 -3.1110520362854004 

